In [1]:
!pip install --upgrade pip setuptools wheel
!pip install qiskit-aer --prefer-binary --no-cache-dir
!pip install qiskit --prefer-binary --no-cache-dir

  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.0 MB/s eta 0:00:00
Using cached setuptools-80.9.0-py3-none-any.whl (1.2 MB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.2.0
    Uninstalling setuptools-75.2.0:
      Successfully uninstalled setuptools-75.2.0
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 157.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 251.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [qiskit-aer]


In [19]:
# Install Qiskit
!pip install qiskit qiskit-aer --quiet

# Imports
from math import gcd
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import numpy as np

# Step 1: User input
N = int(input("Enter a composite number N to factor (e.g., 15, 21): "))
coprimes = [a for a in range(2, N) if gcd(a, N) == 1]
print(f"\nValid coprime choices for N = {N}: {coprimes}")
a = int(input("Select a value for 'a' from the list above: "))
if a not in coprimes:
    raise ValueError("Invalid coprime selected.")

# Step 2: Modular exponentiation circuit (for N = 15 or 21)
def controlled_modular_mult(a, N):
    qc = QuantumCircuit(4)
    for i in range(4):
        if pow(a, 2**i, N) % 2 == 1:
            qc.x(i)
    return qc.to_gate(label=f"U_{a} mod {N}").control()

# Step 3: Quantum Phase Estimation
def qpe(a, N, n_count=3):
    qc = QuantumCircuit(n_count + 4, n_count)
    qc.x(n_count + 3)  # Initialize target qubit to |1>
    qc.h(range(n_count))  # Hadamard on counting qubits

    # Controlled-U operations
    for i in range(n_count):
        gate = controlled_modular_mult(pow(a, 2**i, N), N)
        qc.append(gate, [i] + list(range(n_count, n_count + 4)))

    # Inverse QFT
    for i in range(n_count // 2):
        qc.swap(i, n_count - i - 1)
    for j in range(n_count):
        for m in range(j):
            qc.cp(-np.pi / float(2 ** (j - m)), m, j)
        qc.h(j)

    qc.measure(range(n_count), range(n_count))
    return qc

# Step 4: Run simulation
qc = qpe(a, N)
simulator = AerSimulator()
compiled = transpile(qc, simulator)
result = simulator.run(compiled).result()
counts = result.get_counts()
plot_histogram(counts)

# Step 5: Classical post-processing
def get_order_classically(a, N):
    for r in range(2, N):
        if pow(a, r, N) == 1:
            return r
    return None

r = get_order_classically(a, N)
print(f"\nEstimated order r of a = {a} mod N = {N} is: {r}")

if r and r % 2 == 0:
    x = pow(a, r // 2)
    factor1 = gcd(x - 1, N)
    factor2 = gcd(x + 1, N)
    if factor1 != 1 and factor1 != N:
        print(f"\n🎉 Non-trivial factors of {N} are: {factor1} and {factor2}")
    else:
        print("\n⚠️ Order is even but failed to find non-trivial factors. Try another 'a'.")
else:
    print("\n⚠️ Order is odd or undefined. Try another 'a'.")

Enter a composite number N to factor (e.g., 15, 21): 21

Valid coprime choices for N = 21: [2, 4, 5, 8, 10, 11, 13, 16, 17, 19, 20]
Select a value for 'a' from the list above: 13

Estimated order r of a = 13 mod N = 21 is: 2

🎉 Non-trivial factors of 21 are: 3 and 7
